# prepare the data
* trainA, trainB will be paired as X, y
* testA, testB will also be paired as X, y

In [ ]:
import tensorflow as tf
import os

# Define the data directory
data_dir = '/kaggle/input/facades-dataset'

# Define the image size and batch size
IMG_SIZE = 256
BATCH_SIZE = 1

# Get the file paths for the train and test data
train_a_paths = tf.data.Dataset.list_files(os.path.join(data_dir, 'trainA', '*.jpg'))
train_b_paths = tf.data.Dataset.list_files(os.path.join(data_dir, 'trainB', '*.jpg'))
test_a_paths = tf.data.Dataset.list_files(os.path.join(data_dir, 'testA', '*.jpg'))
test_b_paths = tf.data.Dataset.list_files(os.path.join(data_dir, 'testB', '*.jpg'))

# Combine the train and test data into pairs
train_pairs = tf.data.Dataset.zip((train_b_paths, train_a_paths))
test_pairs = tf.data.Dataset.zip((test_b_paths, test_a_paths))

# Define a function to load and preprocess the images
def load_image(image_file):
    # Read the image file
    image = tf.io.read_file(image_file)
    # Decode the image to a tensor
    image = tf.image.decode_jpeg(image)
    # Resize the image
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Normalize the image to [-1, 1]
    image = (image / 127.5) - 1
    return image

# Load and preprocess the training images
train_dataset = train_pairs.map(lambda x, y: (load_image(x), load_image(y)))

# Load and preprocess the testing images
test_dataset = test_pairs.map(lambda x, y: (load_image(x), load_image(y)))

# Shuffle and batch the training data
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)

# Print the number of training and testing examples
print("Number of training examples: ", tf.data.experimental.cardinality(train_dataset).numpy())
print("Number of testing examples: ", tf.data.experimental.cardinality(test_dataset).numpy())

# define generator


In [3]:
def generator():
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])

    # Encoder
    down1 = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    down2 = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down1)
    down3 = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down2)
    down4 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down3)
    down5 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down4)
    down6 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down5)
    down7 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down6)
    down8 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu', 
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down7)

    # Decoder
    up1 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(down8)
    up1 = tf.keras.layers.Concatenate()([up1, down7])
    up2 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up1)
    up2 = tf.keras.layers.Concatenate()([up2, down6])
    up3 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up2)
    up3 = tf.keras.layers.Concatenate()([up3, down5])
    up4 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up3)
    up4 = tf.keras.layers.Concatenate()([up4, down4])
    up5 = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up4)
    up5 = tf.keras.layers.Concatenate()([up5, down3])
    up6 = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up5)
    up6 = tf.keras.layers.Concatenate()([up6, down2])
    up7 = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up6)
    up7 = tf.keras.layers.Concatenate()([up7, down1])
    outputs = tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(up7)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

# define discriminator

In [4]:
def discriminator():
    inputs = tf.keras.layers.Input(shape=[256, 256, 6])

    # Convolutional layers
    conv1 = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='LeakyReLU')(inputs)
    conv2 = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='LeakyReLU',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(conv1)
    conv3 = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='LeakyReLU',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(conv2)
    conv4 = tf.keras.layers.Conv2D(512, 4, strides=1, padding='same', activation='LeakyReLU',
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(conv3)
    outputs = tf.keras.layers.Conv2D(1, 4, strides=1, padding='same', activation='sigmoid',
                                       kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(conv4)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

# define BinaryCrossentropy losses

In [5]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)
    return total_gen_loss

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss


# define Adam optimizers

In [6]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# define training steps

In [7]:
import matplotlib.pyplot as plt

@tf.function
def train_step(input_image, target):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_real_output = discriminator(tf.concat([input_image, target], axis=-1), training=True)
        disc_generated_output = discriminator(tf.concat([input_image, gen_output], axis=-1), training=True)

        gen_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

def fit(train_ds, epochs, test_ds):
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))

        for input_image, target in train_ds:
            train_step(input_image, target)

        for example_input, example_target in test_ds.take(1):
            generate_images(generator, tf.expand_dims(example_input, axis=0), tf.expand_dims(example_target, axis=0), epoch)

def generate_images(model, test_input, tar, epoch):
    prediction = model(test_input, training=True)
    plt.figure(figsize=(15, 15))
    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))

# do the magic

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 32
LAMBDA = 100
epochs = 150

generator = generator()
discriminator = discriminator()

fit(train_dataset, epochs, test_dataset)